In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from pathlib import Path
from typing import *
import torch
import torch.optim as optim
import numpy as np
import pandas as pd
from functools import partial
from overrides import overrides

from allennlp.data import Instance
from allennlp.data.token_indexers import TokenIndexer
from allennlp.data.tokenizers import Token
from allennlp.nn import util as nn_util

In [3]:
class Config(dict):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        for k, v in kwargs.items():
            setattr(self, k, v)
    
    def set(self, key, val):
        self[key] = val
        setattr(self, key, val)
        
config = Config(
    testing=True,
    seed=1,
    batch_size=64,
    lr=3e-4,
    epochs=2,
    hidden_sz=64,
    max_seq_len=100, # necessary to limit memory usage
    max_vocab_size=100000,
)

In [4]:
from allennlp.common.checks import ConfigurationError

In [5]:
USE_GPU = torch.cuda.is_available()

In [6]:
DATA_ROOT = Path("../data") / "jigsaw"

Set random seed manually to replicate results

In [7]:
torch.manual_seed(config.seed)

# Load Data

In [8]:
from allennlp.data.vocabulary import Vocabulary
from allennlp.data.dataset_readers import DatasetReader

### Prepare dataset

In [9]:
label_cols = ["toxic", "severe_toxic", "obscene",
              "threat", "insult", "identity_hate"]

In [10]:
from allennlp.data.fields import TextField, MetadataField, ArrayField

class JigsawDatasetReader(DatasetReader):
    def __init__(self, tokenizer: Callable[[str], List[str]]=lambda x: x.split(),
                 token_indexers: Dict[str, TokenIndexer] = None,
                 max_seq_len: Optional[int]=config.max_seq_len) -> None:
        super().__init__(lazy=False)
        self.tokenizer = tokenizer
        self.token_indexers = token_indexers or {"tokens": SingleIdTokenIndexer()}
        self.max_seq_len = max_seq_len

    @overrides
    def text_to_instance(self, tokens: List[Token], id: str,
                         labels: np.ndarray) -> Instance:
        sentence_field = TextField(tokens, self.token_indexers)
        fields = {"tokens": sentence_field}
        
        id_field = MetadataField(id)
        fields["id"] = id_field
        
        label_field = ArrayField(array=labels)
        fields["label"] = label_field

        return Instance(fields)
    
    @overrides
    def _read(self, file_path: str) -> Iterator[Instance]:
        df = pd.read_csv(file_path)
        if config.testing: df = df.head(1000)
        for i, row in df.iterrows():
            yield self.text_to_instance(
                [Token(x) for x in self.tokenizer(row["comment_text"])],
                row["id"], row[label_cols].values,
            )

### Prepare token handlers

We will use the spacy tokenizer here

In [11]:
from allennlp.data.tokenizers.word_splitter import SpacyWordSplitter
from allennlp.data.token_indexers import SingleIdTokenIndexer

# the token indexer is responsible for mapping tokens to integers
token_indexer = SingleIdTokenIndexer()

def tokenizer(x: str):
    return [w.text for w in
            SpacyWordSplitter(language='en_core_web_sm', 
                              pos_tags=False).split_words(x)[:config.max_seq_len]]

In [12]:
reader = JigsawDatasetReader(
    tokenizer=tokenizer,
    token_indexers={"tokens": token_indexer}
)

In [13]:
train_ds, test_ds = (reader.read(DATA_ROOT / fname) for fname in ["train.csv", "test_proced.csv"])
val_ds = None

267it [00:02, 94.93it/s]
251it [00:01, 172.26it/s]


In [14]:
len(train_ds)

267

In [15]:
train_ds[:10]

In [16]:
vars(train_ds[0].fields["tokens"])

{'tokens': [Explanation,
  Why,
  the,
  edits,
  made,
  under,
  my,
  username,
  Hardcore,
  Metallica,
  Fan,
  were,
  reverted,
  ?,
  They,
  were,
  n't,
  vandalisms,
  ,,
  just,
  closure,
  on,
  some,
  GAs,
  after,
  I,
  voted,
  at,
  New,
  York,
  Dolls,
  FAC,
  .,
  And,
  please,
  do,
  n't,
  remove,
  the,
  template,
  from,
  the,
  talk,
  page,
  since,
  I,
  'm,
  retired,
  now.89.205.38.27],
 '_token_indexers': {'tokens': <allennlp.data.token_indexers.single_id_token_indexer.SingleIdTokenIndexer at 0x1a27b07400>},
 '_indexed_tokens': None,
 '_indexer_name_to_indexed_token': None}

### Prepare vocabulary

In [17]:
vocab = Vocabulary.from_instances(train_ds, max_vocab_size=config.max_vocab_size)

01/29/2019 19:57:00 - INFO - allennlp.data.vocabulary -   Fitting token dictionary from dataset.
100%|██████████| 267/267 [00:00<00:00, 11635.95it/s]


### Prepare iterator

The iterator is responsible for batching the data and preparing it for input into the model. We'll use the BucketIterator that batches text sequences of smilar lengths together.

In [18]:
from allennlp.data.iterators import BucketIterator

In [19]:
iterator = BucketIterator(batch_size=config.batch_size, 
                          sorting_keys=[("tokens", "num_tokens")],
                         )

We need to tell the iterator how to numericalize the text data. We do this by passing the vocabulary to the iterator. This step is easy to forget so be careful! 

In [20]:
iterator.index_with(vocab)

### Read sample

In [21]:
batch = next(iter(iterator(train_ds)))

In [22]:
batch

{'tokens': {'tokens': tensor([[ 131,  264,   21,  ...,    0,    0,    0],
          [  74,  203,   24,  ...,    0,    0,    0],
          [   5,   85,   26,  ...,    0,    0,    0],
          ...,
          [   5,  103, 1068,  ...,    0,    0,    0],
          [2972,  622, 1099,  ...,    0,    0,    0],
          [   5, 3301,    8,  ...,    0,    0,    0]])},
 'id': ['0029541a38c523a0',
  '003d77a20601cec1',
  '006fda507acd9769',
  '00173958f46763a2',
  '00a5394e626e72c6',
  '005e2ae8f864f76c',
  '0060c5c9030b2d14',
  '0095756047a71716',
  '0000997932d777bf',
  '0061b075244dd234',
  '002d6c9d9f85e81f',
  '001735f961a23fc4',
  '000113f07ec002fd',
  '0029b87aa9c7dc4a',
  '00070ef96486d6f9',
  '007bc29766a43e3c',
  '004f5608984d99f1',
  '000b08c464718505',
  '007bbfa4da2bc32d',
  '00537730daf8c5f1',
  '008f22e7b58e559b',
  '009b3b15f1ada72f',
  '004f981460421bdf',
  '000f35deef84dc4a',
  '002f0e29c60807b1',
  '003dbd1b9b354c1f',
  '004f6dbe69f3545d',
  '00733f0a4a58cf42',
  '0057b7710cb5e

In [23]:
batch["tokens"]["tokens"]

tensor([[ 131,  264,   21,  ...,    0,    0,    0],
        [  74,  203,   24,  ...,    0,    0,    0],
        [   5,   85,   26,  ...,    0,    0,    0],
        ...,
        [   5,  103, 1068,  ...,    0,    0,    0],
        [2972,  622, 1099,  ...,    0,    0,    0],
        [   5, 3301,    8,  ...,    0,    0,    0]])

In [24]:
batch["tokens"]["tokens"].shape

torch.Size([64, 84])

# Prepare Model

In [25]:
import torch
import torch.nn as nn
import torch.optim as optim

In [26]:
from allennlp.modules.seq2vec_encoders import Seq2VecEncoder, PytorchSeq2VecWrapper
from allennlp.nn.util import get_text_field_mask
from allennlp.models import Model
from allennlp.modules.text_field_embedders import TextFieldEmbedder

class BaselineModel(Model):
    def __init__(self, word_embeddings: TextFieldEmbedder,
                 encoder: Seq2VecEncoder,
                 out_sz: int=len(label_cols)):
        super().__init__(vocab)
        self.word_embeddings = word_embeddings
        self.encoder = encoder
        self.projection = nn.Linear(self.encoder.get_output_dim(), out_sz)
        self.loss = nn.BCEWithLogitsLoss()
        
    def forward(self, tokens: Dict[str, torch.Tensor],
                id: Any, label: torch.Tensor) -> torch.Tensor:
        mask = get_text_field_mask(tokens)
        embeddings = self.word_embeddings(tokens)
        state = self.encoder(embeddings, mask)
        class_logits = self.projection(state)
        
        output = {"class_logits": class_logits}
        output["loss"] = self.loss(class_logits, label)

        return output

### Prepare embeddings

In [27]:
from allennlp.modules.token_embedders import Embedding
from allennlp.modules.text_field_embedders import BasicTextFieldEmbedder

token_embedding = Embedding(num_embeddings=config.max_vocab_size + 2,
                            embedding_dim=300, padding_index=0)
# the embedder maps the input tokens to the appropriate embedding matrix
word_embeddings: TextFieldEmbedder = BasicTextFieldEmbedder({"tokens": token_embedding})

In [28]:
from allennlp.modules.seq2vec_encoders import PytorchSeq2VecWrapper
encoder: Seq2VecEncoder = PytorchSeq2VecWrapper(nn.LSTM(word_embeddings.get_output_dim(),
                                                        config.hidden_sz, bidirectional=True, batch_first=True))

Notice how simple and modular the code for initializing the model is. All the complexity is delegated to each component.

In [29]:
model = BaselineModel(
    word_embeddings, 
    encoder, 
)

In [30]:
if USE_GPU: model.cuda()
else: model

# Basic sanity checks

In [31]:
batch = nn_util.move_to_device(batch, 0 if USE_GPU else -1)

In [32]:
tokens = batch["tokens"]
labels = batch

In [33]:
tokens

{'tokens': tensor([[ 131,  264,   21,  ...,    0,    0,    0],
         [  74,  203,   24,  ...,    0,    0,    0],
         [   5,   85,   26,  ...,    0,    0,    0],
         ...,
         [   5,  103, 1068,  ...,    0,    0,    0],
         [2972,  622, 1099,  ...,    0,    0,    0],
         [   5, 3301,    8,  ...,    0,    0,    0]])}

In [34]:
mask = get_text_field_mask(tokens)
mask

tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])

In [35]:
embeddings = model.word_embeddings(tokens)
state = model.encoder(embeddings, mask)
class_logits = model.projection(state)
class_logits

tensor([[-0.0372,  0.0801, -0.0296,  0.0281,  0.0153, -0.0244],
        [-0.0387,  0.0801, -0.0318,  0.0281,  0.0164, -0.0250],
        [-0.0384,  0.0804, -0.0303,  0.0254,  0.0170, -0.0253],
        [-0.0393,  0.0794, -0.0324,  0.0276,  0.0162, -0.0238],
        [-0.0388,  0.0807, -0.0317,  0.0281,  0.0175, -0.0239],
        [-0.0397,  0.0785, -0.0307,  0.0275,  0.0172, -0.0237],
        [-0.0382,  0.0805, -0.0304,  0.0266,  0.0164, -0.0243],
        [-0.0376,  0.0781, -0.0321,  0.0286,  0.0144, -0.0258],
        [-0.0392,  0.0809, -0.0318,  0.0294,  0.0155, -0.0242],
        [-0.0386,  0.0801, -0.0310,  0.0295,  0.0176, -0.0244],
        [-0.0380,  0.0806, -0.0310,  0.0264,  0.0162, -0.0255],
        [-0.0396,  0.0808, -0.0306,  0.0273,  0.0169, -0.0257],
        [-0.0389,  0.0808, -0.0314,  0.0292,  0.0173, -0.0251],
        [-0.0398,  0.0805, -0.0308,  0.0264,  0.0161, -0.0255],
        [-0.0376,  0.0790, -0.0300,  0.0289,  0.0143, -0.0261],
        [-0.0393,  0.0804, -0.0301,  0.0

In [36]:
model(**batch)

{'class_logits': tensor([[-0.0372,  0.0801, -0.0296,  0.0281,  0.0153, -0.0244],
         [-0.0387,  0.0801, -0.0318,  0.0281,  0.0164, -0.0250],
         [-0.0384,  0.0804, -0.0303,  0.0254,  0.0170, -0.0253],
         [-0.0393,  0.0794, -0.0324,  0.0276,  0.0162, -0.0238],
         [-0.0388,  0.0807, -0.0317,  0.0281,  0.0175, -0.0239],
         [-0.0397,  0.0785, -0.0307,  0.0275,  0.0172, -0.0237],
         [-0.0382,  0.0805, -0.0304,  0.0266,  0.0164, -0.0243],
         [-0.0376,  0.0781, -0.0321,  0.0286,  0.0144, -0.0258],
         [-0.0392,  0.0809, -0.0318,  0.0294,  0.0155, -0.0242],
         [-0.0386,  0.0801, -0.0310,  0.0295,  0.0176, -0.0244],
         [-0.0380,  0.0806, -0.0310,  0.0264,  0.0162, -0.0255],
         [-0.0396,  0.0808, -0.0306,  0.0273,  0.0169, -0.0257],
         [-0.0389,  0.0808, -0.0314,  0.0292,  0.0173, -0.0251],
         [-0.0398,  0.0805, -0.0308,  0.0264,  0.0161, -0.0255],
         [-0.0376,  0.0790, -0.0300,  0.0289,  0.0143, -0.0261],
         

In [37]:
loss = model(**batch)["loss"]

In [38]:
loss

tensor(0.6964, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)

In [39]:
loss.backward()

# Train

In [41]:
optimizer = optim.Adam(model.parameters(), lr=config.lr)

In [42]:
from allennlp.training.trainer import Trainer

trainer = Trainer(
    model=model,
    optimizer=optimizer,
    iterator=iterator,
    train_dataset=train_ds,
    cuda_device=0 if USE_GPU else -1,
    num_epochs=config.epochs,
)

In [43]:
metrics = trainer.train()

01/29/2019 19:57:05 - INFO - allennlp.training.trainer -   Beginning training.
01/29/2019 19:57:05 - INFO - allennlp.training.trainer -   Epoch 0/1
01/29/2019 19:57:05 - INFO - allennlp.training.trainer -   Peak CPU memory usage MB: 315.695104
01/29/2019 19:57:05 - INFO - allennlp.training.trainer -   Training
loss: 0.6929 ||: 100%|██████████| 5/5 [00:08<00:00,  1.54s/it]
01/29/2019 19:57:14 - INFO - allennlp.training.trainer -                     Training |  Validation
01/29/2019 19:57:14 - INFO - allennlp.training.trainer -   loss          |     0.693  |       N/A
01/29/2019 19:57:14 - INFO - allennlp.training.trainer -   cpu_memory_MB |   315.695  |       N/A
01/29/2019 19:57:14 - INFO - allennlp.training.trainer -   Epoch duration: 00:00:08
01/29/2019 19:57:14 - INFO - allennlp.training.trainer -   Estimated training time remaining: 0:00:08
01/29/2019 19:57:14 - INFO - allennlp.training.trainer -   Epoch 1/1
01/29/2019 19:57:14 - INFO - allennlp.training.trainer -   Peak CPU memory